<a href="https://colab.research.google.com/github/path-0f-misantrope/Intensive-armatura/blob/main/%D0%BF%D1%80%D0%B8%D0%BA%D0%BE%D0%BB%D1%8E%D1%88%D0%BA%D0%B81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:


import copy
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#импорты
maindf = pd.read_excel('/content/train.xlsx')
gruzDF = pd.read_excel('/content/Грузоперевозки.xlsx')
materials = pd.read_excel('/content/Данные рынка стройматериалов.xlsx')
indexLME = pd.read_excel('/content/Индекс LME.xlsx')
makropokazateli = pd.read_excel('/content/Макропоказатели.xlsx')
toplivo =  pd.read_excel('/content/Топливо.xlsx')
metall = pd.read_excel('/content/Показатели рынка металла.xlsx')
surye  = pd.read_excel('/content/Цены на сырье.xlsx')
CHMF = pd.read_csv('/content/CHMF Акции.csv')
MAGN = pd.read_csv('/content/MAGN Акции.csv')
NLMK = pd.read_csv('/content/NLMK Акции.csv')
#загрузка датасетов

#чтоб удобно было
dfs = {
    "maindf": maindf,
    "gruzDF": gruzDF,
    "materials": materials,
    "indexLME": indexLME,
    "makropokazateli": makropokazateli,
    "toplivo": toplivo,
    "metall": metall,
    "surye": surye,
    "CHMF": CHMF,
    "MAGN": MAGN,
    "NLMK": NLMK
}

for df_name, df in dfs.items():
  print(f"DataFrame: {df_name}")
  df.info()
  print("-" * 50)


DataFrame: maindf
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   dt                400 non-null    datetime64[ns]
 1   Цена на арматуру  400 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 6.4 KB
--------------------------------------------------
DataFrame: gruzDF
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302 entries, 0 to 301
Data columns (total 2 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   dt                               302 non-null    datetime64[ns]
 1   Индекс стоимости грузоперевозок  302 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 4.8 KB
--------------------------------------------------
DataFrame: materials
<class 'pandas.core.frame.D

везде все класс и круто кроме индксов мле и акций

In [11]:
''' убирает из доп данных колонки слабо заполненые '''
def process_dataframe(df):
    threshold = 0.8
    nans_count = df.isna().sum()
    to_drop = nans_count[nans_count / len(df) > threshold].index
    df.drop(to_drop, axis=1, inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df


dfs_to_process = ["NLMK", "MAGN", "CHMF"]
for df_name in dfs_to_process:
  if df_name in dfs:
    dfs[df_name] = process_dataframe(dfs[df_name])





<ipython-input-11-04ddc39e4331>:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill', inplace=True)


In [12]:

# конвертир the 'цена' в индекс мле в флоат
indexLME['цена'] = pd.to_numeric(indexLME['цена'], errors='coerce')


In [13]:
#ну просто чекал
dfs_to_check = ["NLMK", "MAGN", "CHMF", "indexLME"]

for df_name in dfs_to_check:
    if df_name in dfs:
        print(f"DataFrame: {df_name}")
        dfs[df_name].info()
        print("-" * 50)
    else:
        print(f"Warning: DataFrame '{df_name}' not found in the 'dfs' dictionary.")


DataFrame: NLMK
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      1053 non-null   object 
 1   Price     1053 non-null   float64
 2   Open      1053 non-null   float64
 3   High      1053 non-null   float64
 4   Low       1053 non-null   float64
 5   Vol.      1053 non-null   object 
 6   Change %  1053 non-null   object 
dtypes: float64(4), object(3)
memory usage: 57.7+ KB
--------------------------------------------------
DataFrame: MAGN
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Дата    1053 non-null   object
 1   Цена    1053 non-null   object
 2   Откр.   1053 non-null   object
 3   Макс.   1053 non-null   object
 4   Мин.    1053 non-null   object
 5   Объём   1053 non-null   object
 6   Изм. %  

In [27]:
def clean_numeric_dataframe(df):
    """
    Функция принимает DFи преобразует числовые
   убирает запятые (разделители касарей).
   преобразует к в косарь, 'M' в лимоны, 'B' в лярды.
  убирает '%', процентные значения в числовые.
    конвертирует все обработанные колонки в флоат.
    ваще это только для акций нужно, но там короче 2 на англе, и надо "дата" на "date" поменять и еще раз запустить
    """
    df["Дата"] = pd.to_datetime(df["Дата"], format="%d.%m.%Y")
    df_clean = df.copy()

    for col in df_clean.columns:
        if df_clean[col].dtype == "object":

            df_clean[col] = df_clean[col].str.replace(",", "", regex=True)


            df_clean[col] = df_clean[col].str.replace("K", "*1000", regex=True)
            df_clean[col] = df_clean[col].str.replace("M", "*1000000", regex=True)
            df_clean[col] = df_clean[col].str.replace("B", "*1000000000", regex=True)


            df_clean[col] = df_clean[col].str.rstrip("%")

            try:
                df_clean[col] = df_clean[col].apply(pd.eval).astype(float)
            except:
                pass #скип если чет не так

    return df_clean
#CHMF = clean_numeric_dataframe(CHMF)
MAGN = clean_numeric_dataframe(MAGN)
#NLMK = clean_numeric_dataframe(NLMK)

In [28]:
CHMF.info()
print(MAGN.info())
print(NLMK.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1054 entries, 0 to 1053
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   Date      1054 non-null   datetime64[ns]
 1   Price     1054 non-null   float64       
 2   Open      1054 non-null   float64       
 3   High      1054 non-null   float64       
 4   Low       1054 non-null   float64       
 5   Vol.      1054 non-null   float64       
 6   Change %  1054 non-null   float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 57.8 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1053 entries, 0 to 1052
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Дата    1053 non-null   datetime64[ns]
 1   Цена    1053 non-null   float64       
 2   Откр.   1053 non-null   float64       
 3   Макс.   1053 non-null   float64       
 4   Мин.    1053 non-null   float64       
 5   О

In [30]:


#конвертация всех временнных сталбов к единому формату
def convert_datetime_columns(df):
    for col in df.columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            df[col] = df[col].dt.strftime('%d.%m.%Y')
    return df


for df_name, df in dfs.items():
    dfs[df_name] = convert_datetime_columns(df)
